In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import snntorch as snn
from snntorch import surrogate
from snntorch import spikegen

from smogoutils import get_all_data_adj, compute_spiking_node_representation as csnr

In [2]:
# class SimpleSNN(nn.Module):
#     def __init__(self, input_size, hidden_size, output_size):
#         super(SimpleSNN, self).__init__()
#         self.fc1 = nn.Linear(input_size, hidden_size)
#         self.bn1 = nn.BatchNorm1d(hidden_size)
#         self.lif1 = snn.Leaky(beta=0.9, spike_grad=surrogate.fast_sigmoid())
#         self.fc2 = nn.Linear(hidden_size, output_size)
#         self.bn2 = nn.BatchNorm1d(output_size)
#         self.lif2 = snn.Leaky(beta=0.9, spike_grad=surrogate.fast_sigmoid())
        
#         # Weight initialization
#         nn.init.xavier_uniform_(self.fc1.weight)
#         nn.init.xavier_uniform_(self.fc2.weight)

#     def forward(self, x):
#         spk_rec = []
#         mem_rec = []
#         mem1 = self.lif1.init_leaky()
#         mem2 = self.lif2.init_leaky()

#         for step in range(x.size(0)):
#             cur1 = self.fc1(x[step])
#             cur1 = self.bn1(cur1)
#             spk1, mem1 = self.lif1(cur1, mem1)
#             cur2 = self.fc2(spk1)
#             cur2 = self.bn2(cur2)
#             spk2, mem2 = self.lif2(cur2, mem2)
#             spk_rec.append(spk2)
#             mem_rec.append(mem2)

#         return torch.stack(spk_rec), torch.stack(mem_rec)

from smogomodels import SMOGOGCN as SimpleSNN


In [3]:
data_folder = "BRCA"
view_list = [1, 2, 3]
num_class = 5

(
    data_tr_list,
    data_trte_list,
    trte_idx,
    labels_trte,
    labels_tr_tensor,
    onehot_labels_tr_tensor,
    adj_tr_list,
    adj_te_list,
    dim_list,
) = get_all_data_adj(
    data_folder,
    view_list,
    num_class,
)

/Users/gabriquaranta/repos/GU01-bioinspired-spatial-data/SMOGONET/smogoutils.py:120: UserWarning: torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, shape, dtype=, device=). (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:623.)
  return sparse_tensortype(indices, values, x.size())


In [4]:
K = 3
num_steps = 100

H_tr_v1 = csnr(data_tr_list[0], adj_tr_list[0], K, num_steps)
H_tr_v2 = csnr(data_tr_list[1], adj_tr_list[1], K, num_steps)
H_tr_v3 = csnr(data_tr_list[2], adj_tr_list[2], K, num_steps)

H_trte_v1 = csnr(data_trte_list[0], adj_te_list[0], K, num_steps)
H_trte_v2 = csnr(data_trte_list[1], adj_te_list[1], K, num_steps)
H_trte_v3 = csnr(data_trte_list[2], adj_te_list[2], K, num_steps)

H_te_v1=H_trte_v1
H_te_v2=H_trte_v2
H_te_v3=H_trte_v3

labels_tr=labels_tr_tensor
labels_trte_tensor = torch.tensor(labels_trte)
labels_te=labels_trte_tensor[trte_idx['te']]

In [5]:
print(H_tr_v1.shape, H_tr_v2.shape, H_tr_v3.shape)
print(H_trte_v1.shape, H_trte_v2.shape, H_trte_v3.shape)
print(labels_tr.shape, labels_trte_tensor.shape, labels_te.shape)

torch.Size([100, 612, 1000]) torch.Size([100, 612, 1000]) torch.Size([100, 612, 503])
torch.Size([100, 875, 1000]) torch.Size([100, 875, 1000]) torch.Size([100, 875, 503])
torch.Size([612]) torch.Size([875]) torch.Size([263])


In [6]:
hidden_size = 50  # neurons in the hidden layer
output_size = 5  # output classes
num_steps = 100  # time steps
num_epochs = 100  

In [7]:
#snn1
inputs=H_tr_v1
targets=labels_tr

snn1 = SimpleSNN(inputs.shape[2], hidden_size, output_size)
optimizer = optim.Adam(snn1.parameters(), lr=0.01)
loss_fn = nn.CrossEntropyLoss()

for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs, _ = snn1(inputs)
    outputs = outputs.mean(dim=0)  # average over time steps

    loss = loss_fn(outputs, targets)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}')

Epoch 10/100, Loss: 1.1196715831756592
Epoch 20/100, Loss: 1.0645732879638672
Epoch 30/100, Loss: 1.038413643836975
Epoch 40/100, Loss: 1.0139058828353882
Epoch 50/100, Loss: 0.9938926696777344
Epoch 60/100, Loss: 0.9787880778312683
Epoch 70/100, Loss: 0.966449499130249
Epoch 80/100, Loss: 0.9573761224746704
Epoch 90/100, Loss: 0.9503357410430908
Epoch 100/100, Loss: 0.9451058506965637


In [8]:
#snn2
inputs=H_tr_v2
targets=labels_tr

snn2 = SimpleSNN(inputs.shape[2], hidden_size, output_size)
optimizer = optim.Adam(snn2.parameters(), lr=0.01)
loss_fn = nn.CrossEntropyLoss()

for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs, _ = snn2(inputs)
    outputs = outputs.mean(dim=0)  # average over time steps

    loss = loss_fn(outputs, targets)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}')

Epoch 10/100, Loss: 1.1896507740020752
Epoch 20/100, Loss: 1.1206048727035522
Epoch 30/100, Loss: 1.0867419242858887
Epoch 40/100, Loss: 1.0621848106384277
Epoch 50/100, Loss: 1.0436464548110962
Epoch 60/100, Loss: 1.0302352905273438
Epoch 70/100, Loss: 1.015643835067749
Epoch 80/100, Loss: 0.9956215023994446
Epoch 90/100, Loss: 0.9785406589508057
Epoch 100/100, Loss: 0.9685261845588684


In [9]:
#snn3
inputs=H_tr_v3
targets=labels_tr
snn3 = SimpleSNN(inputs.shape[2], hidden_size, output_size)
optimizer = optim.Adam(snn3.parameters(), lr=0.01)
loss_fn = nn.CrossEntropyLoss()

for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs, _ = snn3(inputs)
    outputs = outputs.mean(dim=0)  # average over time steps

    loss = loss_fn(outputs, targets)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}')

Epoch 10/100, Loss: 1.210241675376892
Epoch 20/100, Loss: 1.1344162225723267
Epoch 30/100, Loss: 1.0945888757705688
Epoch 40/100, Loss: 1.0698878765106201
Epoch 50/100, Loss: 1.0501703023910522
Epoch 60/100, Loss: 1.0348299741744995
Epoch 70/100, Loss: 1.022658109664917
Epoch 80/100, Loss: 1.0117430686950684
Epoch 90/100, Loss: 1.0030217170715332
Epoch 100/100, Loss: 0.9953594207763672


In [10]:
idx_te = trte_idx['te']
labels_te = labels_trte_tensor[idx_te]

In [11]:
# eval snn1
inputs=H_te_v1
targets=labels_trte_tensor

outputs1s, _ = snn1(inputs)
outputs1 = outputs1s.mean(dim=0) 
_, predicted1 = torch.max(outputs1, 1)

test_acc1 = (predicted1 == targets).sum().item() / targets.size(0)
print(f'Test accuracy of SNN1 trte: {test_acc1}')

# eval snn1 te
outputs1_te=outputs1[idx_te]
predicted1_te=predicted1[idx_te]

test_acc1_te = (predicted1_te == labels_te).sum().item() / labels_te.size(0)
print(f'Test accuracy of SNN1 te: {test_acc1_te}')

Test accuracy of SNN1 trte: 0.8697142857142857
Test accuracy of SNN1 te: 0.8631178707224335


In [12]:
# eval snn2 trte
inputs=H_te_v2
# targets=labels_trte_tensor

outputs2s, _ = snn2(inputs)
outputs2 = outputs2s.mean(dim=0)
_, predicted2 = torch.max(outputs2, 1)

test_acc2 = (predicted2 == targets).sum().item() / targets.size(0)
print(f'Test accuracy of SNN2 trte: {test_acc2}')

# eval snn2 te
outputs2_te=outputs2[idx_te]
predicted2_te=predicted2[idx_te]

test_acc2_te = (predicted2_te == labels_te).sum().item() / labels_te.size(0)
print(f'Test accuracy of SNN2 te: {test_acc2_te}')

Test accuracy of SNN2 trte: 0.8011428571428572
Test accuracy of SNN2 te: 0.7642585551330798


In [13]:
# eval snn3
inputs=H_te_v3
# targets=labels_trte_tensor

outputs3s, _ = snn3(inputs)
outputs3 = outputs3s.mean(dim=0)
_, predicted3 = torch.max(outputs3, 1)

test_acc3 = (predicted3 == targets).sum().item() / targets.size(0)
print(f'Test accuracy of SNN3: {test_acc3}')

# eval snn3 te
outputs3_te=outputs3[idx_te]
predicted3_te=predicted3[idx_te]

test_acc3_te = (predicted3_te == labels_te).sum().item() / labels_te.size(0)
print(f'Test accuracy of SNN3 te: {test_acc3_te}')

Test accuracy of SNN3: 0.7737142857142857
Test accuracy of SNN3 te: 0.7262357414448669


In [14]:
# combined avg trte
outputs_avg = (outputs1 + outputs2 + outputs3) / 3
_, predicted_avg = torch.max(outputs_avg, 1)

test_acc_avg = (predicted_avg == targets).sum().item() / targets.size(0)
print(f'Test accuracy of SNN_avg trte: {test_acc_avg}')

# combined avg te
outputs_avg_te = (outputs1_te + outputs2_te + outputs3_te) / 3
_, predicted_avg_te = torch.max(outputs_avg_te, 1)

test_acc_avg_te = (predicted_avg_te == labels_te).sum().item() / labels_te.size(0)
print(f'Test accuracy of SNN_avg te: {test_acc_avg_te}')

Test accuracy of SNN_avg trte: 0.8617142857142858
Test accuracy of SNN_avg te: 0.8250950570342205


In [15]:
output_spikes=[outputs1s, outputs2s, outputs3s]

print(output_spikes[0].shape, output_spikes[1].shape, output_spikes[2].shape)

print(outputs1s[0], outputs2s[0], outputs3s[0])  # cant be usend to create cross correlatoin matrix directly?

torch.Size([100, 875, 5]) torch.Size([100, 875, 5]) torch.Size([100, 875, 5])
tensor([[1., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 1., 1., 0., 0.],
        ...,
        [0., 0., 0., 0., 1.],
        [0., 1., 0., 0., 1.],
        [0., 0., 0., 1., 0.]], grad_fn=<SelectBackward0>) tensor([[0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 1.],
        ...,
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1.]], grad_fn=<SelectBackward0>) tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 1., 0.],
        ...,
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1.]], grad_fn=<SelectBackward0>)


In [16]:
in_list=[outputs1.detach(), outputs2.detach(), outputs3.detach()]
num_view=3
num_cls=5

for i in range(num_view):
    in_list[i] = torch.sigmoid(in_list[i])


x = torch.reshape(
    torch.matmul(in_list[0].unsqueeze(-1), in_list[1].unsqueeze(1)),
    (-1, pow(num_cls, 2), 1),
)


x = torch.reshape(
    torch.matmul(x, in_list[2].unsqueeze(1)),
    (-1, pow(num_cls, 3), 1),
)
print("after outer product", x.shape)

vcdn_feat = torch.reshape(x, (-1, pow(num_cls, num_view)))

print(vcdn_feat.shape)

vcdn_input_spike_encoding= spikegen.rate(vcdn_feat,100)

print(vcdn_input_spike_encoding.shape)


after outer product torch.Size([875, 125, 1])
torch.Size([875, 125])
torch.Size([100, 875, 125])


In [17]:
# class SimpleSNN_VCDN(nn.Module):
#     def __init__(self, input_size, hidden_size, output_size):
#         super(SimpleSNN_VCDN, self).__init__()
#         self.fc1 = nn.Linear(input_size, hidden_size)
#         self.bn1 = nn.BatchNorm1d(hidden_size)
#         self.lif1 = snn.Leaky(beta=0.9, spike_grad=surrogate.fast_sigmoid())
#         self.fc2 = nn.Linear(hidden_size, output_size)
#         self.bn2 = nn.BatchNorm1d(output_size)
#         self.lif2 = snn.Leaky(beta=0.9, spike_grad=surrogate.fast_sigmoid())
        
#         # Weight initialization
#         nn.init.xavier_uniform_(self.fc1.weight)
#         nn.init.xavier_uniform_(self.fc2.weight)

#     def forward(self, x):
#         spk_rec = []
#         mem_rec = []
#         mem1 = self.lif1.init_leaky()
#         mem2 = self.lif2.init_leaky()

#         for step in range(x.size(0)):
#             cur1 = self.fc1(x[step])
#             cur1 = self.bn1(cur1)
#             spk1, mem1 = self.lif1(cur1, mem1)
#             cur2 = self.fc2(spk1)
#             cur2 = self.bn2(cur2)
#             spk2, mem2 = self.lif2(cur2, mem2)
#             spk_rec.append(spk2)
#             mem_rec.append(mem2)

#         return torch.stack(spk_rec), torch.stack(mem_rec)
    
from smogomodels import SMOGOVCDN as SimpleSNN_VCDN

In [18]:
inputs_vcdn=vcdn_input_spike_encoding
targets_vcdn=labels_trte_tensor
num_epochs=100

snn_vcdn = SimpleSNN_VCDN(inputs_vcdn.shape[2], 50, 5)
optimizer = optim.Adam(snn_vcdn.parameters(), lr=0.01)
loss_fn = nn.CrossEntropyLoss()

for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs, _ = snn_vcdn(inputs_vcdn)
    outputs = outputs.mean(dim=0)  # average over time steps

    loss = loss_fn(outputs, targets_vcdn)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}')

outputs_vcdn, _ = snn_vcdn(inputs_vcdn)
accuracy=(torch.argmax(outputs_vcdn.mean(dim=0), dim=1) == targets_vcdn).sum().item() / targets_vcdn.size(0)
print(f'Accuracy of VCDN: {accuracy}')

accuracy_te=(torch.argmax(outputs_vcdn.mean(dim=0)[idx_te], dim=1) == labels_te).sum().item() / labels_te.size(0)
print(f'Accuracy of VCDN te: {accuracy_te}')

Epoch 10/100, Loss: 1.1490455865859985
Epoch 20/100, Loss: 1.084875226020813
Epoch 30/100, Loss: 1.0583386421203613
Epoch 40/100, Loss: 1.0466694831848145
Epoch 50/100, Loss: 1.0397849082946777
Epoch 60/100, Loss: 1.0357261896133423
Epoch 70/100, Loss: 1.032678484916687
Epoch 80/100, Loss: 1.0299639701843262
Epoch 90/100, Loss: 1.0276787281036377
Epoch 100/100, Loss: 1.026084303855896
Accuracy of VCDN: 0.9017142857142857
Accuracy of VCDN te: 0.8935361216730038
